In [1]:
from pyspark.sql import SparkSession,DataFrame
import os
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.types import *
import io
import time
from pyspark.sql import Row
from pyspark.sql.functions import lit, unix_timestamp 

In [2]:
local=True
# spark.rpc.message.maxSize if for write large csv file. The default value is 128, here we set it to 1024
if local:
    spark = SparkSession \
    .builder.master("local[4]") \
    .appName("SparkKafkaWriter") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1') \
    .getOrCreate()
else: 
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkKafkaWriter") \
    .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1') \
    .getOrCreate()

In [3]:
spark.sparkContext.stop()

In [26]:
print(spark.sparkContext.version)

3.1.1


In [15]:
! kubectl get pods

I0929 09:07:23.992270    3939 request.go:655] Throttling request took 1.167895229s, request: GET:https://kubernetes.default/apis/acme.cert-manager.io/v1alpha2?timeout=32s
NAME                               READY   STATUS    RESTARTS   AGE
flume-test-agent-df8c5b944-vtjbx   1/1     Running   0          9d
jupyter-266220-5bf4b859f8-wfkz6    1/1     Running   0          25h
kafka-server-0                     1/1     Running   0          9d
kafka-server-1                     1/1     Running   0          9d
kafka-server-2                     1/1     Running   0          9d
kafka-server-zookeeper-0           1/1     Running   0          9d
rstudio-625080-8d7b9fbfb-wdmlw     1/1     Running   0          21h


In [28]:
! kubectl get pods | grep sparkarrow | awk '{print $1}' | xargs kubectl delete pods

I0928 13:59:34.345375    1999 request.go:655] Throttling request took 1.166803404s, request: GET:https://kubernetes.default/apis/acme.cert-manager.io/v1alpha3?timeout=32s
I0928 13:59:38.077155    2034 request.go:655] Throttling request took 1.181243355s, request: GET:https://kubernetes.default/apis/acme.cert-manager.io/v1beta1?timeout=32s
pod "sparkarrowcompression-77efdb7c26eb6e0b-exec-5" deleted
pod "sparkarrowcompression-77efdb7c26eb6e0b-exec-6" deleted
pod "sparkarrowcompression-77efdb7c26eb6e0b-exec-7" deleted
pod "sparkarrowcompression-77efdb7c26eb6e0b-exec-8" deleted


In [7]:
data = [("xiaomi", "2007"),
        ("xiaomi 3G","2008"),
      ("xiaomi 3GS","2009"),
      ("xiaomi 4","2010"),
      ("xiaomi 4S","2011"),
      ("xiaomi 5","2012"),
      ("xiaomi 8","2014"),
      ("xiaomi 3GS","2009"),
        ("xiaomi 10","2017")
       ]


data1 = [("iphone", "2107"),
        ("iphone 3G","2108"),
      ("iphone 3GS","2109"),
      ("iphone 4","2110"),
      ("iphone 4S","2111"),
      ("iphone 5","2112"),
      ("iphone 8","2114"),
      ("iphone 3GS","2109"),
        ("iphone 10","2117")
       ]
df = spark.createDataFrame(data1).toDF("key","value")


In [8]:
df.show()
df.printSchema()

+----------+-----+
|       key|value|
+----------+-----+
|    iphone| 2107|
| iphone 3G| 2108|
|iphone 3GS| 2109|
|  iphone 4| 2110|
| iphone 4S| 2111|
|  iphone 5| 2112|
|  iphone 8| 2114|
|iphone 3GS| 2109|
| iphone 10| 2117|
+----------+-----+

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [9]:
df1=df.withColumn("toto",lit(unix_timestamp()))
df1.show()

+----------+-----+----------+
|       key|value|      toto|
+----------+-----+----------+
|    iphone| 2107|1632922275|
| iphone 3G| 2108|1632922275|
|iphone 3GS| 2109|1632922275|
|  iphone 4| 2110|1632922275|
| iphone 4S| 2111|1632922275|
|  iphone 5| 2112|1632922275|
|  iphone 8| 2114|1632922275|
|iphone 3GS| 2109|1632922275|
| iphone 10| 2117|1632922275|
+----------+-----+----------+



In [10]:
# When we write the data frame to kafka, only the key, value column get read by the consumer
# The kafka console consumer can only read the value column. Need to find out why.
df.write \
  .format("kafka") \
  .option("kafka.bootstrap.servers","kafka-server.user-pengfei.svc.cluster.local:9092") \
  .option("topic","test-topic") \
  .save()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



+----------+-----+----------+---------+------+-----------------------+-------------+
|key       |value|topic     |partition|offset|timestamp              |timestampType|
+----------+-----+----------+---------+------+-----------------------+-------------+
|iphone 3GS|2009 |test_topic|0        |0     |2021-09-28 14:38:06.326|0            |
|iphone 8  |2014 |test_topic|0        |1     |2021-09-28 14:38:06.326|0            |
|iphone 4  |2010 |test_topic|0        |2     |2021-09-28 14:38:06.356|0            |
|iphone    |2007 |test_topic|0        |3     |2021-09-28 14:38:06.326|0            |
|iphone 4S |2011 |test_topic|0        |4     |2021-09-28 14:38:06.326|0            |
|iphone 10 |2017 |test_topic|0        |5     |2021-09-28 14:38:06.357|0            |
|iphone 3G |2008 |test_topic|0        |6     |2021-09-28 14:38:06.357|0            |
|iphone 5  |2012 |test_topic|0        |7     |2021-09-28 14:38:06.357|0            |
|xiaomi    |2007 |test_topic|0        |8     |2021-09-28 15:18:02